In [9]:
import PIL
import imageio
import diffusers
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from ipywidgets import interactive, fixed
from IPython.display import Image, display, clear_output


import torch
from torchmetrics.image.inception import InceptionScore
from torchmetrics.image.fid import FrechetInceptionDistance

from datasets import load_dataset
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from diffusers import (
    DiffusionPipeline,
    StableDiffusionPipeline,
    EulerDiscreteScheduler,
    PNDMScheduler,
    LMSDiscreteScheduler,
)
from diffusers import AutoPipelineForText2Image
import ipywidgets as widgets

# Manually enabling the ipywidgets
# (in case the widgets still do not get displayed correctly, have a look at https://ipywidgets.readthedocs.io/en/latest/user_install.html)
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run script troubleshoot

Jupyter command `jupyter-nbextension` not found.


In [8]:
def generate_images(prompt, num_images, scheduler, steps):
    pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")

    # Load the LoRA weights for the model
    pipeline.load_lora_weights("Checkpoint_L1/checkpoint-250", weight_name="pytorch_lora_weights.safetensors")

    # Set the scheduler (this part is a placeholder, please adapt to your needs)
    if scheduler == "DDIM":
        pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")
    # Add other schedulers as needed

    # Generate images based on the prompt
    images = []
    for _ in range(num_images):
        image = pipeline(prompt, num_inference_steps=steps, guidance_scale=7.5).images[0]
        images.append(image)
    return images


In [10]:
def display_images(prompt, num_images, steps, scheduler):
    images = generate_images(prompt, num_images, scheduler, steps)
    for idx, img in enumerate(images):
        display(img)
        img.save(f"generated_image_{idx+1}.png")

# Create interactive widgets
interactive_plot = interactive(display_images,
                               prompt=widgets.Textarea(value='', placeholder='Enter your prompt...', description='Prompt:'),
                               num_images=widgets.IntSlider(value=1, min=1, max=10, step=1, description='Num Images:'),
                               steps=widgets.IntSlider(value=50, min=1, max=100, step=1, description='Inference Steps:'),
                               scheduler=widgets.Dropdown(options=["DDIM", "PLMS", "Euler", "Euler a", "LMS", "Heun", "DPM2", "DPM2 a", "DPM++", "DPM++ a"], value='DDIM', description='Scheduler:'))

# Display the interactive widgets
display(interactive_plot)



interactive(children=(Textarea(value='', description='Prompt:', placeholder='Enter your prompt...'), IntSlider…